In [11]:
import pandas as pd
import numpy as np

In [12]:
data = pd.read_csv('data_movies.csv', low_memory=False)
data.shape

(50993, 15)

In [13]:
data = data.drop_duplicates(subset=['id'])
data.reset_index(drop=True, inplace=True)

In [14]:
data.head(2)

,id,imdb_id,release_date,title,original_language,genres,popularity,production_companies,production_countries,vote_average,vote_count,overview,runtime,casts,directors
0,862,tt0114709,1995-10-30,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",220.672,"[{'id': 3, 'logo_path': '/1TjvGVDMYsj6JBxOAkUH...","[{'iso_3166_1': 'US', 'name': 'United States o...",8.0,15473,"Led by Woody, Andy's toys live happily in his ...",81.0,"[{'cast_id': 31, 'name': 'Tom Hanks'}, {'cast_...","[{'cast_id': 7879, 'name': 'John Lasseter'}]"
1,8844,tt0113497,1995-12-15,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",15.750,"[{'id': 559, 'logo_path': '/jqWioYeGSyTLuHth01...","[{'iso_3166_1': 'US', 'name': 'United States o...",7.2,9025,When siblings Judy and Peter discover an encha...,104.0,"[{'cast_id': 2157, 'name': 'Robin Williams'}, ...","[{'cast_id': 4945, 'name': 'Joe Johnston'}]"


In [15]:
# Function to convert all strings to lower case and strip names of spaces
def lower_noSpace(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [16]:
# Apply clean_data function to your features.
features = ['title']

for feature in features:
    data[feature] = data[feature].apply(lower_noSpace)

In [17]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['casts', 'directors', 'genres']
for feature in features:
    data[feature] = data[feature].apply(literal_eval)

In [18]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [19]:
def get_mainActor(x):
    for i in x:
        return i['name']

In [20]:
# Define new main_actor, casts, genres features that are in a suitable form.
data['main_actor'] = data['casts'].apply(get_mainActor)

features = ['casts', 'genres', 'directors']
for feature in features:
    data[feature] = data[feature].apply(get_list)

In [21]:
# Print the new features of the first 3 films
data[['title', 'casts', 'main_actor', 'directors', 'genres']].head(3)

,title,casts,main_actor,directors,genres
0,toystory,"[Tom Hanks, Tim Allen, Don Rickles]",Tom Hanks,[John Lasseter],"[Animation, Adventure, Family]"
1,jumanji,"[Robin Williams, Kirsten Dunst, Bradley Pierce]",Robin Williams,[Joe Johnston],"[Adventure, Fantasy, Family]"
2,grumpieroldmen,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Walter Matthau,[Howard Deutch],"[Romance, Comedy]"


In [22]:
# Apply lower_noSpace function to your features.
features = ['casts', 'main_actor', 'directors', 'genres']

for feature in features:
    data[feature] = data[feature].apply(lower_noSpace)

In [23]:
# Print the new features of the first 3 films
data[['title', 'original_language', 'casts', 'main_actor', 'directors', 'genres']].head(3)

,title,original_language,casts,main_actor,directors,genres
0,toystory,en,"[tomhanks, timallen, donrickles]",tomhanks,[johnlasseter],"[animation, adventure, family]"
1,jumanji,en,"[robinwilliams, kirstendunst, bradleypierce]",robinwilliams,[joejohnston],"[adventure, fantasy, family]"
2,grumpieroldmen,en,"[waltermatthau, jacklemmon, ann-margret]",waltermatthau,[howarddeutch],"[romance, comedy]"


In [24]:
data.head(2)

,id,imdb_id,release_date,title,original_language,genres,popularity,production_companies,production_countries,vote_average,vote_count,overview,runtime,casts,directors,main_actor
0,862,tt0114709,1995-10-30,toystory,en,"[animation, adventure, family]",220.672,"[{'id': 3, 'logo_path': '/1TjvGVDMYsj6JBxOAkUH...","[{'iso_3166_1': 'US', 'name': 'United States o...",8.0,15473,"Led by Woody, Andy's toys live happily in his ...",81.0,"[tomhanks, timallen, donrickles]",[johnlasseter],tomhanks
1,8844,tt0113497,1995-12-15,jumanji,en,"[adventure, fantasy, family]",15.750,"[{'id': 559, 'logo_path': '/jqWioYeGSyTLuHth01...","[{'iso_3166_1': 'US', 'name': 'United States o...",7.2,9025,When siblings Judy and Peter discover an encha...,104.0,"[robinwilliams, kirstendunst, bradleypierce]",[joejohnston],robinwilliams


In [25]:
def create_soup(x):
    return ' '.join(x['casts']) + ' ' +  x['main_actor'] + ' ' + x['original_language'] + ' ' + ' '.join(x['directors']) + ' ' + ' '.join(x['genres']) + ' ' + x['original_language']

In [26]:
# Create a new soup feature
data['soup'] = data.apply(create_soup, axis=1)

In [27]:
# Print the new features of the first 3 films
data[['title','soup']].head(3)

,title,soup
0,toystory,tomhanks timallen donrickles tomhanks en johnl...
1,jumanji,robinwilliams kirstendunst bradleypierce robin...
2,grumpieroldmen,waltermatthau jacklemmon ann-margret waltermat...


In [81]:
data.head(2)

,id,imdb_id,release_date,title,original_language,genres,popularity,production_companies,production_countries,vote_average,vote_count,overview,runtime,casts,directors,main_actor,soup
0,862,tt0114709,1995-10-30,toystory,en,"[animation, adventure, family]",220.672,"[{'id': 3, 'logo_path': '/1TjvGVDMYsj6JBxOAkUH...","[{'iso_3166_1': 'US', 'name': 'United States o...",8.0,15473,"Led by Woody, Andy's toys live happily in his ...",81.0,"[tomhanks, timallen, donrickles]",[johnlasseter],tomhanks,tomhanks timallen donrickles tomhanks en johnl...
1,8844,tt0113497,1995-12-15,jumanji,en,"[adventure, fantasy, family]",15.750,"[{'id': 559, 'logo_path': '/jqWioYeGSyTLuHth01...","[{'iso_3166_1': 'US', 'name': 'United States o...",7.2,9025,When siblings Judy and Peter discover an encha...,104.0,"[robinwilliams, kirstendunst, bradleypierce]",[joejohnston],robinwilliams,robinwilliams kirstendunst bradleypierce robin...


In [82]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
data['score'] = data.apply(weighted_rating, axis=1)

In [85]:
data.sort_values('score', ascending=False).head(2)

,id,imdb_id,release_date,title,original_language,genres,popularity,production_companies,production_countries,vote_average,vote_count,overview,runtime,casts,directors,main_actor,soup,score
314,278,tt0111161,1994-09-23,theshawshankredemption,en,"[drama, crime]",69.679,"[{'id': 97, 'logo_path': '/7znWcbDd4PcJzJUlJxY...","[{'iso_3166_1': 'US', 'name': 'United States o...",8.7,21582,Framed in the 1940s for the double murder of h...,142.0,"[timrobbins, morganfreeman, bobgunton]",[frankdarabont],timrobbins,timrobbins morganfreeman bobgunton timrobbins ...,8.628196
834,238,tt0068646,1972-03-14,thegodfather,en,"[drama, crime]",65.651,"[{'id': 4, 'logo_path': '/gz66EfNoYPqHTYI4q9UE...","[{'iso_3166_1': 'US', 'name': 'United States o...",8.7,16074,"Spanning the years 1945 to 1955, a chronicle o...",175.0,"[marlonbrando, alpacino, jamescaan]",[francisfordcoppola],marlonbrando,marlonbrando alpacino jamescaan marlonbrando e...,8.604408


In [104]:
data.to_csv(r'data_movies_final.csv', index = False)

# Define the Top rated movies 

In [28]:
m = data['vote_count'].quantile(0.90)
C = data['vote_average'].mean()
q_movies = data.copy().loc[data['vote_count'] >= m]

In [29]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [30]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [31]:
q_movies = q_movies.sort_values('score', ascending=False)

In [32]:
q_movies.reset_index(drop=True, inplace=True)

In [33]:
q_movies[['id','title','vote_count', 'vote_average', 'score']].head(20)

,id,title,vote_count,vote_average,score
0,278,theshawshankredemption,21582,8.7,8.628196
1,238,thegodfather,16074,8.7,8.604408
2,424,schindler'slist,12846,8.6,8.485497
3,240,thegodfather:partii,9692,8.6,8.450244
4,155,thedarkknight,27698,8.5,8.447649
5,680,pulpfiction,23270,8.5,8.437919
6,13,forrestgump,22819,8.5,8.436721
7,122,thelordoftherings:thereturnoftheking,19634,8.5,8.426736
8,497,thegreenmile,14012,8.5,8.398455
9,129,spiritedaway,12925,8.5,8.390265


In [22]:
q_movies.to_csv(r'topMovies.csv', index = False)